In [1]:
import csv
import random
from autogluon.tabular import TabularDataset, TabularPredictor
import os
from wikimapper import WikiMapper
import collections
property_label = "P19"
orientation = "objects"
limit = "full"

In [2]:
# load all instances
p50_instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.subjects.P50.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in p50_instances:
        p50_instances[line[0]] = [line[1]]
    else:
        p50_instances[line[0]].append(line[1])

p19_instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.objects.P19.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in p19_instances:
        p19_instances[line[0]] = [line[1]]
    else:
        p19_instances[line[0]].append(line[1])

p39_instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.objects.P39.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in p39_instances:
        p39_instances[line[0]] = [line[1]]
    else:
        p39_instances[line[0]].append(line[1])

p161_instances = {}
tsv_file = open(f'../output/allConstraintsAnalysis_Final/instances/instances.subjects.P161.tsv', encoding="utf-8")
read_tsv = csv.reader(tsv_file, delimiter="\t")
for line in read_tsv:
    if line[0] not in p161_instances:
        p161_instances[line[0]] = [line[1]]
    else:
        p161_instances[line[0]].append(line[1])

p50_objects = set()
tsv_file = open(f'../data/propertiesSplit_final/claims.P50.tsv')
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0] in p50_instances:
        p50_objects.add(line[0])
p50_objects = random.sample(list(p50_objects),250000)
p50_class_map = {}
for cla in p50_objects:
    p50_class_map[cla] = 1

p161_objects = set()
tsv_file = open(f'../data/propertiesSplit_final/claims.P161.tsv')
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[0] in p161_instances:
        p161_objects.add(line[0])
p161_class_map = {}
for cla in p161_objects:
    p161_class_map[cla] = 1
    
p39_objects = set()
tsv_file = open(f'../data/propertiesSplit_final/claims.P39.tsv')
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[2] in p39_instances:
        p39_objects.add(line[2])
p39_class_map = {}
for cla in p39_objects:
    p39_class_map[cla] = 1
    
p19_objects = set()
tsv_file = open(f'../data/propertiesSplit_final/claims.P19.tsv')
read_tsv = csv.reader(tsv_file, delimiter="\t")
next(read_tsv)
for line in read_tsv:
    if line[2] in p19_instances:
        p19_objects.add(line[2])
p19_class_map = {}
for cla in p19_objects:
    p19_class_map[cla] = 1
    
# new_p50_map = {}
# for key in p50_instances:
#     if key in p50_class_map:
#         new_p50_map[key] = p50_instances[key]
# p50_instances = new_p50_map

print(len(p19_class_map))
print(len(p39_class_map))
print(len(p50_class_map))
print(len(p161_class_map))
print(len(p19_instances))
print(len(p39_instances))
print(len(p50_instances))
print(len(p161_instances))

print(p50_instances["Q15719277"])
all_instances = {**p19_instances,**p39_instances,**p161_instances,**p50_instances}
print(all_instances["Q19041414"])

222250
103855
250000
207004
222251
103856
925380
207005
['Q21191270']
['Q19389637']


In [19]:
# map all wikipedia labels to wikidata qnodes
mapper = WikiMapper("../../wikimapper/data/index_enwiki-latest.db")
with open("../embeddings/enwiki_20180420_300d.txt", encoding="utf-8") as in_file, open("../embeddings/wd_embed_from_wp.txt","w",encoding="utf-8",newline='') as out_file:
    tsv_reader = csv.reader(in_file, delimiter=" ")
    tsv_writer = csv.writer(out_file, delimiter = "\t")
    next(tsv_reader)
    for line in tsv_reader:
        if "ENTITY/" in line[0]:
            wpnode = line[0].split("/")[1]
            wdnode = mapper.title_to_id(wpnode)
            if wdnode == None:
                print("no page found for ", wpnode)
            else:
                this_arr = [wdnode]
                for embed in line:
                    if "ENTITY/" not in embed:
                        this_arr.append(embed)
                tsv_writer.writerow(this_arr)

None


In [22]:
# get all qnodes in both transE and wikipedia embeddings
wikipedia_qnodes = set()
transE_qnodes = set()

transE_files = ["P19.objects.transE.embeddings.full.tsv","P39.objects.transE.embeddings.full.tsv","P50.subjects.transE.embeddings.full.tsv","P161.subjects.transE.embeddings.full.tsv"]

for file in transE_files:
    with open("../output/instance_prediction_datasets/250000/"+file, encoding="utf-8") as this_file:
        tsv_reader = csv.reader(this_file, delimiter="\t")
        next(tsv_reader)
        for line in tsv_reader:
            transE_qnodes.add(line[0])
            
print(len(transE_qnodes))

with open("../embeddings/wd_embed_from_wp.txt", encoding="utf-8") as this_file:
    tsv_reader = csv.reader(this_file, delimiter="\t")
    for line in tsv_reader:
        wikipedia_qnodes.add(line[0])

all_qnodes = wikipedia_qnodes.intersection(transE_qnodes)
print(len(all_qnodes))

with open(f"../output/instance_prediction_datasets/250000/overlapped_qnodes.txt", 'wt',encoding="utf-8",newline='') as out_file:
    tsv_writer = csv.writer(out_file)
    for node in all_qnodes:
        tsv_writer.writerow([node])

510789
207629


In [30]:
# make equally sized transE, wikipedia, and combined datasets

transEheader = []
textHeader = ["Qnode"]
bothHeader = ["Qnode"]
for i in range(0,400):
    bothHeader.append("pos"+str(i))
bothHeader.append("Yval")
for i in range(0,300):
    textHeader.append("pos"+str(i))
textHeader.append("Yval")

properties = {"P19":"objects","P39":"objects","P161":"subjects","P50":"subjects"}

for pp in properties:
    
    transEMap = {}
    textMap = {}
    instances = {}
    
    print(pp)
    print(properties[pp])
    
    with open(f"../output/instance_prediction_datasets/250000/{pp}.{properties[pp]}.transE.embeddings.full.tsv", encoding="utf-8") as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        transEheader = next(tsv_reader)
        for line in tsv_reader:
            if line[0] in all_qnodes:
                transEMap[line[0]] = line[1:-1]
                assert len(transEMap[line[0]]) == 100
                instances[line[0]] = line[-1]
                
    with open(f"../embeddings/wd_embed_from_wp.txt", encoding="utf-8") as file:
        tsv_reader = csv.reader(file, delimiter='\t')
        for line in tsv_reader:
            if line[0] in transEMap:
                textMap[line[0]] = line[1:]
                assert len(textMap[line[0]]) == 300, str(len(textMap[line[0]])) + str(textMap[line[0]])
            
    assert len(transEMap) == len(textMap) == len(instances), (len(transEMap), len(textMap), len(instances))
    
    # get ordered list of qnodes
    all_keys = list(transEMap.keys())
    random.shuffle(all_keys)
            
    with open(f"../output/instance_prediction_datasets/250000/{pp}.{properties[pp]}.transE.embeddings.synchronized.tsv", "w", encoding="utf-8", newline='') as transEfile:
        writer = csv.writer(transEfile, delimiter="\t")
        writer.writerow(transEheader)
        for elem in all_keys:
            this_arr = []
            this_arr.append(elem)
            for row in transEMap[elem]:
                this_arr.append(row)
            this_arr.append(instances[elem])
            writer.writerow(this_arr)
    with open(f"../output/instance_prediction_datasets/250000/{pp}.{properties[pp]}.wptext.embeddings.synchronized.tsv", "w", encoding="utf-8", newline='') as textFile:
        writer = csv.writer(textFile, delimiter="\t")
        writer.writerow(textHeader)
        for elem in all_keys:
            this_arr = []
            this_arr.append(elem)
            for row in textMap[elem]:
                this_arr.append(row)
            this_arr.append(instances[elem])
            writer.writerow(this_arr)
    with open(f"../output/instance_prediction_datasets/250000/{pp}.{properties[pp]}.combined.embeddings.synchronized.tsv", "w", encoding="utf-8", newline='') as bothFile:
        writer = csv.writer(bothFile, delimiter="\t")
        writer.writerow(bothHeader)
        for elem in all_keys:
            this_arr = []
            this_arr.append(elem)
            for row in transEMap[elem]:
                this_arr.append(row)
            for row in textMap[elem]:
                this_arr.append(row)
            this_arr.append(instances[elem])
            writer.writerow(this_arr)

P19
objects
P39
objects
P161
subjects
P50
subjects


In [6]:
# make one dataset to rule them all
header = []
files = ["P39.objects.transE.embeddings.full.tsv","P19.objects.transE.embeddings.full.tsv","P50.subjects.transE.embeddings.full.tsv","P161.subjects.transE.embeddings.full.tsv"]
filemap = {"P39":[],"P50":[],"P161":[],"P19":[]}
for file in files:
    print(file)
    with open(f"../output/instance_prediction_datasets/250000/"+file,encoding="utf-8") as in_file:
        reader = csv.reader(in_file, delimiter="\t")
        header = next(reader)
        header = file.split(".")[0]
        for line in reader:
            filemap[header].append(line)
            
for key in filemap:
    print(key)
    print(len(filemap[key]))
    filemap[key] = random.choices(filemap[key], k=50000)

full_list = []
for key in filemap:
    for line in filemap[key]:
        full_list.append(line)
        
random.shuffle(full_list)

with open("../output/instance_prediction_datasets/250000/all.transE.embeddings.full.tsv", "w", encoding="utf-8", newline="") as out_file:
    writer = csv.writer(out_file, delimiter="\t")
    writer.writerow(header)
    writer.writerows(full_list)

P39.objects.transE.embeddings.full.tsv
P19.objects.transE.embeddings.full.tsv
P50.subjects.transE.embeddings.full.tsv
P161.subjects.transE.embeddings.full.tsv
P39
56186
P50
105547
P161
156934
P19
192809


In [3]:
# fit autogluon models
datatype = "combined"
#datatype = "wptext"
#datatype = "transE"
all_data = TabularDataset(f'../output/instance_prediction_datasets/250000/all.transE.embeddings.full.tsv')
print(int(len(all_data)*.75))
train_data = all_data[:int(len(all_data)*.75)]
test_data = all_data[int(len(all_data)*.75):]
save_path = f'../embeddings/models/agModels.all.full.250000'
label = 'Yval'
# print("Summary of class variable: \n", train_data[label].describe())
# predictor = TabularPredictor(label=label, path=save_path).fit(train_data, hyperparameters={'NN':{},'GBM':{},'XT':{},'KNN':{}})

150000


In [4]:
# build test data and evaluate models
save_path = f'../embeddings/models/agModels.all.full.250000'
predictor = TabularPredictor.load(save_path)
y_test = test_data[label]
test_data_nolab = test_data.drop(columns=[label])
print(predictor.get_model_best())
y_pred = predictor.predict(test_data_nolab)
perf = predictor.evaluate_predictions(y_true=y_test, y_pred=y_pred, auxiliary_metrics=True)
#predictor.leaderboard(test_data, silent=True)
print(list(y_test))
print(list(y_pred))

WeightedEnsemble_L2
['Q21191270', 'Q19389637', 'Q11424', 'Q3331189', 'Q506240', 'Q100532807', 'Q43501', 'Q486972', 'Q3331189', 'Q1499928', 'Q7725634', 'Q3331189', 'Q4164871', 'Q13433827', 'Q19389637', 'Q484170', 'Q11424', 'Q4164871', 'Q3957', 'Q28930084', 'Q5153359', 'Q4164871', 'Q2775236', 'Q80096233', 'Q19389637', 'Q294414', 'Q13406463', 'Q11424', 'Q11424', 'Q10870555', 'Q11424', 'Q4164871', 'Q19389637', 'Q100532807', 'Q4164871', 'Q11424', 'Q212971', 'Q51049922', 'Q11424', 'Q47461344', 'Q76514543', 'Q5153359', 'Q4164871', 'Q3331189', 'Q5398426', 'Q47461344', 'Q11424', 'Q532', 'Q11424', 'Q4164871', 'Q11424', 'Q47461344', 'Q4164871', 'Q1259759', 'Q3558970', 'Q11424', 'Q212971', 'Q7725634', 'Q506240', 'Q2590631', 'Q19389637', 'Q19389637', 'Q532', 'Q7725634', 'Q4164871', 'Q4164871', 'Q13406463', 'Q11424', 'Q3331189', 'Q4164871', 'Q187685', 'Q47461344', 'Q11424', 'Q1639634', 'Q1549591', 'Q4164871', 'Q13220204', 'Q532', 'Q19389637', 'Q4164871', 'Q4164871', 'Q1289426', 'Q2555896', 'Q11424',

In [10]:
from collections import Counter

def most_frequent(List):
#     maxlist = []
#     for singlelist in List:
#         for elem in singlelist:
#             maxlist.append(elem)
#     return max(set(maxlist), key = maxlist.count)
    return max(set(List), key = List.count)

incorrect = {}
correct = {}
counts = {}
y_test_list = list(y_test)
y_pred_list = list(y_pred)
for index, value in enumerate(y_test_list):
    prediction = y_pred_list[index]
    instances = all_instances[entries[index]]
    if prediction not in instances:        
        if value in incorrect:
            incorrect[value].append(prediction)
        else:
            incorrect[value] = [prediction]
    else:
        if value in correct:
            correct[value].append(prediction)
        else:
            correct[value] = [prediction]
for key in incorrect:
    correctcount = 0
    if key in correct:
        correctcount = len(correct[key])
    counts[key] = len(incorrect[key])/((len(incorrect[key])+correctcount))
filtered_counted = {}
for key in counts:
    correctcount = 0
    if key in correct:
        correctcount = len(correct[key])
    if (len(incorrect[key])+correctcount) >= 10:
        filtered_counted[key] = counts[key]
print({k:v for k, v in sorted(filtered_counted.items(), key=lambda item: item[1], reverse=True)})
for key in incorrect:
    print(key, most_frequent(incorrect[key]))
#print(correct)
# occurence_count = Counter(y_pred_list)
# print(occurence_count)
occurence_count = Counter(y_test_list)
print(occurence_count)

{'Q193495': 0.6666666666666666, 'Q17537576': 0.5833333333333334, 'Q15078955': 0.5789473684210527, 'Q591041': 0.38461538461538464, 'Q14406742': 0.375, 'Q277759': 0.30303030303030304, 'Q1667921': 0.2857142857142857, 'Q399811': 0.2727272727272727, 'Q1004': 0.2727272727272727, 'Q87167': 0.2727272727272727, 'Q2831984': 0.2631578947368421, 'Q355567': 0.25, 'Q482': 0.23529411764705882, 'Q2630741': 0.23529411764705882, 'Q56019': 0.23076923076923078, 'Q100341898': 0.2222222222222222, 'Q15978299': 0.2222222222222222, 'Q3191695': 0.2222222222222222, 'Q2989398': 0.21428571428571427, 'Q89487741': 0.2, 'Q3024240': 0.19230769230769232, 'Q19833170': 0.18181818181818182, 'Q58632367': 0.18181818181818182, 'Q1002697': 0.18181818181818182, 'Q1980247': 0.18181818181818182, 'Q28640': 0.1791044776119403, 'Q1279564': 0.17073170731707318, 'Q3297186': 0.16666666666666666, 'Q30749496': 0.15, 'Q333291': 0.14814814814814814, 'Q49084': 0.1417910447761194, 'Q5185279': 0.13966480446927373, 'Q2179958': 0.1333333333333

In [6]:
# calculate top-k metric
entries = list(test_data["Qnode"])
probabilities = predictor.predict_proba(test_data_nolab)
probabilities_dict = probabilities.to_dict()
# print(len(probabilities_dict["Q7777570"]))
# print(len(entries))
# score = 0
# total = 0
# ytest_list = list(y_test)
# for index, entry in enumerate(entries):
#     classes = {}
#     for cla in probabilities_dict:
#         classes[cla] = probabilities_dict[cla][index+150000]
#     mc = collections.Counter(classes).most_common()
#     ordered = [item[0] for item in mc]
#     done = False
#     for elem in ordered:
#         if done == False:
#             for instance in all_instances[entry]:
#                 if elem == instance:
#                     ti = ordered.index(instance)+1
#                     score += 1/ti
#                     done = True
#                     break
#     total += 1
# print(score)
# print(total)
# print(score/total)